# step 1: Predefined CLass

In [1]:
class Propaganda:
    NEGATIVE = "NEGATIVE"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self,sentence,SUBJprop):
        self.sentence = sentence
        self.SUBJprop = SUBJprop
        self.propaganda = self.get_propaganda()

    def get_propaganda(self):
        if self.SUBJprop <=3:
            return Propaganda.NEGATIVE
        else:
            return Propaganda.POSITIVE

# step 2: Load Data

In [45]:
import pandas as pd
import numpy as np
reviews = []
data  = pd.read_excel('Data/Dataset.xlsx', engine='openpyxl')
df = pd.DataFrame(data, columns = ['Sentence','SUBJprop'])
df.dropna()


# iterate elements of attribute "Sentence" and "SUBJprop" and push to the array "reviews"

for index, row in df.iterrows():
    sentence = row['Sentence']
    prop = row['SUBJprop']
    reviews.append(Review(sentence,prop))


print(reviews[0].sentence)
print(reviews[0].propaganda)




miley and liam fighting false rumors swirl that theyre in a feud over a supposed prenup
NEGATIVE


# step 3: Prep Data (split into train and test set)

In [46]:
from sklearn.model_selection import train_test_split
training, test = train_test_split(reviews, train_size=0.7, random_state = 42 ) 
# len(training)
print(len(test)) # 70% of traning set

developmentSet, testSet = train_test_split(test, train_size=0.5, random_state = 42 ) 
print(len(testSet)) # 15% of test set
print(len(developmentSet)) # 15% of development set
print(developmentSet[0].sentence)

991
496
495
chibok girls kidnapper jailed for 15 years  kano nigeria a man involved in the 2014 kidnapping of more than 200 schoolgirls from chibok in northeast nigeria has been jailed for 15 years the government confirmed on tuesday


# step 4: Seperate the attribute, originally our array has text and score. we want them to be a seperate array


In [47]:
train_sentence = [ x.sentence for x  in training]     # return just text
train_propaganda = [x.propaganda for x in training]   # return just sentiment

test_sentence = [x.sentence for x in testSet]
test_propaganda = [x.propaganda for x in testSet]

development_sentence = [x.sentence for x in developmentSet]
development_propaganda = [x.propaganda for x in developmentSet]



# step 4: Bag Of Words

In [50]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_sentence) # return a matrix of either 0 or 1 # train_x is our text
test_x_vectors = vectorizer.transform(test_sentence)
# Todo:
# transform development set to vectors
print(train_sentence[0])
print(train_x_vectors[0].toarray())
print(test_x_vectors[0].toarray())

bey and serena have been running in the same circles since the 22time grammy winner was a member of destiny’s child
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]


# step 5: Classification SVM

In [90]:
from sklearn import svm

clf_svm = svm.SVC(kernel='rbf',C=1)
clf_svm.fit(train_x_vectors, train_propaganda)

clf_svm.predict(test_x_vectors[1])

array(['NEGATIVE'], dtype='<U8')

# step 5: Decision Tree 

In [91]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_propaganda)

clf_dec.predict(test_x_vectors[1])

array(['NEGATIVE'], dtype='<U8')

# step 5: Logistic Regression

In [92]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_propaganda)

clf_log.predict(test_x_vectors[1])

array(['NEGATIVE'], dtype='<U8')

# step 6: Evaluation

In [93]:
# Mean Accuracy
# For Support Vector Machine
print(clf_svm.score(test_x_vectors,test_propaganda))
# For Decision Tree
print(clf_dec.score(test_x_vectors,test_propaganda))
# For Logistic Regression
print(clf_log.score(test_x_vectors,test_propaganda))

0.9737903225806451
0.9193548387096774
0.9717741935483871


In [95]:
# F1 Scores
from sklearn.metrics import f1_score

# For Support Vector Machine
print(f1_score(test_propaganda, clf_svm.predict(test_x_vectors),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))
# trash for negative and neutral labels

# For Support Decision Tree
print(f1_score(test_propaganda,clf_dec.predict(test_x_vectors),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))


# For Logistic Regression
print(f1_score(test_propaganda,clf_log.predict(test_x_vectors),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))


[0.         0.98672114]
[0.04761905 0.95789474]
[0.         0.98568507]


In [71]:
import numpy as np
# check number of positive and negative classes in training set
print(train_propaganda.count(Propaganda.POSITIVE))
print(train_propaganda.count(Propaganda.NEGATIVE))

# check number of positive and negative classes in test set
y = np.array(test_propaganda)
number_of_NEGATIVE = (y == "NEGATIVE").sum()
number_of_POSITIVE = (y == "POSITIVE").sum()
print(number_of_NEGATIVE)
print(number_of_POSITIVE)

# check number of positive and negative classes in development set

y = np.array(development_propaganda)
Dnumber_of_NEGATIVE = (y == "NEGATIVE").sum()
Dnumber_of_POSITIVE = (y == "POSITIVE").sum()
print(Dnumber_of_NEGATIVE)
print(Dnumber_of_POSITIVE)



75
2236
483
13
478
17


In [72]:
test_set =['this is advertisement','bad book do not buy','horrible waste of time']
new_test = vectorizer.transform(test_set)
clf_svm.predict(new_test)

array(['NEGATIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

# step 7: Tuning our model (with Grid Search)

In [74]:
from sklearn.model_selection import GridSearchCV

# tuned = svm.SVC()
parameters = {'kernel':('linear','rbf'),'C':(1,4,8,16,32,64)}
svc = svm.SVC()
clf = GridSearchCV(svc,parameters,cv=10)
clf.fit(train_x_vectors,train_propaganda)
clf.cv_results_


{'mean_fit_time': array([0.27320447, 0.67022581, 0.20351579, 0.55123885, 0.20574124,
        0.47740765, 0.20137808, 0.41298549, 0.19411895, 0.56466937,
        0.22485716, 0.43851888]),
 'std_fit_time': array([0.04711192, 0.1968799 , 0.00476683, 0.10113094, 0.00541937,
        0.09228566, 0.01168189, 0.00889045, 0.01774756, 0.11364768,
        0.05027391, 0.08015903]),
 'mean_score_time': array([0.02650533, 0.06406367, 0.02055156, 0.05172002, 0.02098153,
        0.04960811, 0.02010186, 0.04560652, 0.01941109, 0.08710849,
        0.02798283, 0.05384669]),
 'std_score_time': array([0.00559316, 0.02832779, 0.00059292, 0.00234023, 0.00149918,
        0.00655486, 0.00112681, 0.00111799, 0.00215659, 0.04548177,
        0.02646646, 0.02205971]),
 'param_C': masked_array(data=[1, 1, 4, 4, 8, 8, 16, 16, 32, 32, 64, 64],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, False],
        fill_value='?',
             dtype=object),

In [77]:
import pandas as pd

df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.273204,0.047112,0.026505,0.005593,1,linear,"{'C': 1, 'kernel': 'linear'}",0.956897,0.961039,0.952381,0.961039,0.965368,0.965368,0.965368,0.952381,0.974026,0.974026,0.962789,0.007280,7
1,0.670226,0.196880,0.064064,0.028328,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.965517,0.969697,0.969697,0.969697,0.969697,0.969697,0.965368,0.965368,0.965368,0.965368,0.967547,0.002150,1
2,0.203516,0.004767,0.020552,0.000593,4,linear,"{'C': 4, 'kernel': 'linear'}",0.943966,0.948052,0.952381,0.956710,0.961039,0.965368,0.965368,0.948052,0.974026,0.974026,0.958899,0.010254,8
3,0.551239,0.101131,0.051720,0.002340,4,rbf,"{'C': 4, 'kernel': 'rbf'}",0.965517,0.969697,0.969697,0.969697,0.969697,0.969697,0.965368,0.965368,0.965368,0.965368,0.967547,0.002150,1
4,0.205741,0.005419,0.020982,0.001499,8,linear,"{'C': 8, 'kernel': 'linear'}",0.948276,0.948052,0.948052,0.956710,0.961039,0.956710,0.965368,0.948052,0.974026,0.974026,0.958031,0.009859,10
5,0.477408,0.092286,0.049608,0.006555,8,rbf,"{'C': 8, 'kernel': 'rbf'}",0.965517,0.969697,0.969697,0.969697,0.969697,0.969697,0.965368,0.965368,0.965368,0.965368,0.967547,0.002150,1
6,0.201378,0.011682,0.020102,0.001127,16,linear,"{'C': 16, 'kernel': 'linear'}",0.948276,0.956710,0.943723,0.956710,0.961039,0.961039,0.965368,0.952381,0.974026,0.969697,0.958897,0.008898,9
7,0.412985,0.008890,0.045607,0.001118,16,rbf,"{'C': 16, 'kernel': 'rbf'}",0.961207,0.969697,0.969697,0.969697,0.969697,0.969697,0.965368,0.965368,0.965368,0.965368,0.967116,0.002836,4
8,0.194119,0.017748,0.019411,0.002157,32,linear,"{'C': 32, 'kernel': 'linear'}",0.948276,0.956710,0.952381,0.956710,0.952381,0.948052,0.965368,0.948052,0.965368,0.978355,0.957165,0.009354,11
9,0.564669,0.113648,0.087108,0.045482,32,rbf,"{'C': 32, 'kernel': 'rbf'}",0.961207,0.969697,0.969697,0.965368,0.969697,0.969697,0.965368,0.965368,0.965368,0.965368,0.966683,0.002738,5


In [78]:
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,linear,0.962789
1,1,rbf,0.967547
2,4,linear,0.958899
3,4,rbf,0.967547
4,8,linear,0.958031
5,8,rbf,0.967547
6,16,linear,0.958897
7,16,rbf,0.967116
8,32,linear,0.957165
9,32,rbf,0.966683


In [79]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}